In [4]:
import math
import numpy as np
import pandas as pd
import scipy.stats as scs
import statsmodels.api as sm
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.integrate import quad


In [2]:
def simulate_gbm():
    #model parameters
    S0=2600 #initial S&P500 index level
    T=1
    vol=0.2
    
    #simulate parameters
    np.random.seed(10000)
    gbm_dates=pd.DatetimeIndex(start='13-04-2018',end='12-04-2019',freq='B')
    M = len(gbm_dates) #time steps
    I=1 #index level paths
    dt = 1/252 
    df=1 #discount factor is 1 because interest rate is 0
    
    #stock price paths
    rand = np.random.standard_normal((M,I))#random numbers
    S = np.zeros_like(rand) #stock matrix
    S[0]=S0 #initial index
    for t in range(1,M): 
        S[t]=S[t-1]*np.exp((-vol**2/2)*dt+vol*rand[t]*math.sqrt(dt))
    gbm=pd.DataFrame(S[:,0],index=gbm_dates,columns=['index'])
    return gbm


    
    
    

In [5]:
def dN(X):
    #probability density function of standard normal random variable x
    return math.exp(-0.5*x**2)/math.sqrt(2*math.pi)

def N(d):
    #cumulative density function of standard normal random variable x
    return quad(lambda x:dN(x),-20,d,limit=50)[0]

def d1f(St,K,t,T,r,sigma):
    #B-S d1 function
    d1 = (math.log(St/K)+(r+0.5*sigma**2)*(T-t))/(sigma*math.sqrt(T-t))
    return d1

def BSM_delta(St,K,t,T,r,sigma):
    #BS delta of European call
    d1 = d1f(St,K,t,T,r,sigma)
    delta = N(d1)
    return delta
    
def BSM_gamma(St,K,t,T,r,sigma):
    #BS gamma of European call
    d1 = d1f(St,K,t,T,r,sigma)
    gamma = dN(d1)/(St*sigma*math.sqrt(T-t))
    return gamma

def BSM_theta(St,K,t,T,r,sigma):
    d1 = d1f(St,K,t,T,r,sigma)
    d2 = d1 - sigma*math.sqrt(T-t)
    theta = -(St*dN(d1)*sigma/(2*math.sqrt(T-t))+r*K*math.exp(-r*(T-t)*N(d2)))
    return theta
